In [23]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import os
import zipfile
import requests

In [24]:
# Step 1: Download the dataset
url = 'https://madm.dfki.de/files/sentinel/EuroSAT.zip'
dataset_path = '/content/EuroSAT.zip'

In [25]:
# Download the dataset with SSL verification disabled
if not os.path.exists(dataset_path):
    print("Downloading dataset...")
    response = requests.get(url, stream=True, verify=False)  # Disable SSL verification
    with open(dataset_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)

In [26]:
# Step 2: Extract the dataset
extract_path = '/content/EuroSAT'
if not os.path.exists(extract_path):
    print("Extracting dataset...")
    with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

In [27]:
# Define the path to the extracted dataset
data_path = os.path.join(extract_path, '2750')

In [28]:
# Verify the extracted dataset
if not os.path.exists(data_path):
    print(f"The path {data_path} does not exist. Please check the extraction.")
else:
    # Print the subdirectories to verify the structure
    subdirectories = os.listdir(data_path)
    print("Subdirectories in the data path:", subdirectories)

    # Step 3: Set up the data generator
    data_generator = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        validation_split=0.2
    )


Subdirectories in the data path: ['River', 'SeaLake', 'PermanentCrop', 'Industrial', 'Forest', 'Residential', 'Pasture', 'AnnualCrop', 'Highway', 'HerbaceousVegetation']


In [29]:
# Generate the training data
train_data = data_generator.flow_from_directory(
data_path,
target_size=(64, 64),
batch_size=8,
class_mode='categorical',
subset='training',
classes=['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake']
)


# Generate the validation data
test_data = data_generator.flow_from_directory(
data_path,
target_size=(64, 64),
batch_size=1,
subset='validation',
class_mode='categorical',
classes=['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake']
)

Found 21600 images belonging to 10 classes.
Found 5400 images belonging to 10 classes.


In [30]:
CNN = keras.Sequential([
        keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Conv2D(64, (3, 3), activation='relu'),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Conv2D(64, (3, 3), activation='relu'),
        keras.layers.Flatten(),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(10,activation='softmax')

])

In [31]:
CNN.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
CNN.fit(train_data, validation_data=test_data, epochs=20)

Epoch 1/20
2700/2700 [==============================] - 60s 22ms/step - loss: 1.2853 - accuracy: 0.5149 - val_loss: 0.8812 - val_accuracy: 0.6950
Epoch 2/20
2700/2700 [==============================] - 59s 22ms/step - loss: 0.7717 - accuracy: 0.7187 - val_loss: 0.7324 - val_accuracy: 0.7294
Epoch 3/20
2700/2700 [==============================] - 58s 21ms/step - loss: 0.6187 - accuracy: 0.7767 - val_loss: 0.5518 - val_accuracy: 0.7976
Epoch 4/20
2700/2700 [==============================] - 58s 22ms/step - loss: 0.5280 - accuracy: 0.8107 - val_loss: 0.5625 - val_accuracy: 0.8013
Epoch 5/20
2700/2700 [==============================] - 59s 22ms/step - loss: 0.4812 - accuracy: 0.8300 - val_loss: 0.5211 - val_accuracy: 0.8091
Epoch 6/20
2700/2700 [==============================] - 58s 21ms/step - loss: 0.4397 - accuracy: 0.8440 - val_loss: 0.5756 - val_accuracy: 0.7981
Epoch 7/20
2700/2700 [==============================] - 57s 21ms/step - loss: 0.3959 - accuracy: 0.8615 - val_loss: 0.4731 -

In [ ]:
CNN.fit(train_data, validation_data=test_data, epochs=10)

Epoch 1/10
2700/2700 [==============================] - 58s 21ms/step - loss: 0.1867 - accuracy: 0.9346 - val_loss: 0.5732 - val_accuracy: 0.8487
Epoch 2/10
2700/2700 [==============================] - 58s 21ms/step - loss: 0.1847 - accuracy: 0.9345 - val_loss: 0.5416 - val_accuracy: 0.8576
Epoch 3/10
2700/2700 [==============================] - 58s 22ms/step - loss: 0.1728 - accuracy: 0.9386 - val_loss: 0.5924 - val_accuracy: 0.8450
Epoch 4/10
2700/2700 [==============================] - 58s 22ms/step - loss: 0.1737 - accuracy: 0.9418 - val_loss: 0.6430 - val_accuracy: 0.8393
Epoch 5/10
2700/2700 [==============================] - 57s 21ms/step - loss: 0.1688 - accuracy: 0.9420 - val_loss: 0.5642 - val_accuracy: 0.8533
Epoch 6/10
2700/2700 [==============================] - 57s 21ms/step - loss: 0.1617 - accuracy: 0.9449 - val_loss: 0.6208 - val_accuracy: 0.8519
Epoch 7/10
2700/2700 [==============================] - 58s 22ms/step - loss: 0.1550 - accuracy: 0.9484 - val_loss: 0.6250 -

In [32]:
import os

# Define the path to save the model
model_save_path = '/content/drive/My Drive/Test/code/saved_model/my_model'

# Create the directory if it does not exist
os.makedirs(os.path.dirname(model_save_path), exist_ok=True)

# Save the trained model
CNN.save(model_save_path)
print(f"Model saved to {model_save_path}")


Model saved to /content/drive/My Drive/Test/code/saved_model/my_model


In [33]:
# Define the path where the model was saved
model_save_path = '/content/drive/My Drive/Test/code/saved_model/my_model'

# Load the saved model
loaded_model = keras.models.load_model(model_save_path)
print("Model loaded successfully")


Model loaded successfully


In [34]:
def preprocess_image(img_path):
    # Load the image
    img = keras.preprocessing.image.load_img(img_path, target_size=(64, 64))
    # Convert the image to an array
    img_array = keras.preprocessing.image.img_to_array(img)
    # Rescale the image
    img_array = img_array / 255.0
    # Add a batch dimension
    img_array = np.expand_dims(img_array, axis=0)
    return img_array


In [35]:
def predict_image(model, img_path):
  img_array = preprocess_image(img_path)
  predictions = model.predict(img_array)
  predicted_class = np.argmax(predictions, axis=1)
  return predicted_class

In [37]:
import requests
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow import keras
import io

# Function to predict image class
def predict_image(model, img_path):
    try:
        img = Image.open(img_path)
        img = img.resize((64, 64))  # Resize to model's input shape
        img_array = np.array(img) / 255.0  # Normalize image
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
        predictions = model.predict(img_array)
        predicted_class = np.argmax(predictions)
        return predicted_class
    except Exception as e:
        print(f"Error processing image: {e}")
        return None

# Load your saved model
model_save_path = '/content/drive/My Drive/Test/code/saved_model/my_model'
loaded_model = keras.models.load_model(model_save_path)
print("Model loaded successfully.")

# Define the direct download URL from Google Drive https://drive.google.com/file/d/1C9rkh_1uKAzAsZDza2q93UvwKAtqeUZK/view?usp=drive_link
file_id = '1C9rkh_1uKAzAsZDza2q93UvwKAtqeUZK'  # Update with your new file ID
new_image_url = f'https://drive.google.com/uc?export=download&id={file_id}'
new_image_path = '/content/new_image.jpg'

# Mapping of class indices to class names
class_names = ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake']

# Download the image from URL
response = requests.get(new_image_url, stream=True)
if response.status_code == 200:
    # Save the image to local path
    with open(new_image_path, 'wb') as f:
        f.write(response.content)
    print(f"Image downloaded to {new_image_path}")

    # Check if the downloaded file is a valid image
    try:
        img = Image.open(new_image_path)
        img.verify()  # Verify if it's an image
    except Exception as e:
        print(f"Error: {e}")
        print("The downloaded file is not a valid image.")
        exit()

    # Proceed with image prediction
    predicted_class_index = predict_image(loaded_model, new_image_path)
    if predicted_class_index is not None:
        predicted_class_name = class_names[predicted_class_index]
        print(f"Predicted class index: {predicted_class_index}")
        print(f"Predicted class name: {predicted_class_name}")
    else:
        print("Failed to predict image class.")
else:
    print(f"Failed to download image. Status code: {response.status_code}")


Model loaded successfully.
Image downloaded to /content/new_image.jpg
1/1 [==============================] - 0s 85ms/step
Predicted class index: 5
Predicted class name: Pasture
